# Constrained Shape Optimization on inductance

We will allow the red boundary (frontier) to move freely, rather than being constrained by predefined parameters. Furthermore, we will allow the vertical segments connected to the red frontier to shift freely in the vertical direction, providing additional flexibility in the design.

We minimize the magnetic energy within the conductor region while constraining the inductance. To achieve this, we use the augmented Lagrangian algorithm, aiming to reduce losses.

![inductance](assets/inductance-geometry.png "Inductance geometry")

In [ ]:
# !pip install ngsolve --upgrade
# !pip install webgui_jupyter_widgets --upgrade
# !pip install matplotlib --upgrade

from commonSG import create_plots, update_plots, rot  # noqa: F401

import ngsolve as ngs
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle
from netgen.geom2d import EdgeInfo as EI, PointInfo as PI, Solid2d
import numpy as np
import matplotlib.pyplot as plt

## 1 - Meshing

In [ ]:
from netgen.geom2d import SplineGeometry


# Global domain
r = 0.04

# Geometry definition
e = 5e-3
a = 1e-2
ha = 1e-2
ba = 1e-2
d = 1e-2


def gen_mesh(air_gap, maxh=7e-4, debug=False):
    """Gives a triangular mesh"""
    geo = SplineGeometry()
    pnts = [
        (0, air_gap / 2),  # p1
        (a / 2, air_gap / 2),  # p2
        (a / 2, e / 2 + ha / 2),  # p3
        (a / 2 + ba, e / 2 + ha / 2),  # p4
        (a / 2 + ba, air_gap / 2),  # p5
        (a + ba, air_gap / 2),  # p6
        (a + ba, e / 2 + ha / 2 + a / 2),  # p7
        (0, e / 2 + ha / 2 + a / 2),  # p8
        (a + ba, 0),  # p001
        (a + ba, e / 2 + ha / 2),  # p002
        (0, e / 2 + ha / 2),  # p003
        (0, 0),  # p00
        (r, 0),  # p01
        (0, r),  # p02
        (a / 2, 0),  # p03
        (a / 2 + ba, 0),  # p04
        (r, r),  # p05
    ]

    (
        p1,
        p2,
        p3,
        p4,
        p5,
        p6,
        p7,
        p8,
        p001,
        p002,
        p003,
        p00,
        p01,
        p02,
        p03,
        p04,
        p05,
    ) = [geo.AppendPoint(*pnt) for pnt in pnts]

    # List of lines with boundary conditions and domains
    lines = [
        [["line", p1, p2], {"bc": "front", "leftdomain": 5, "rightdomain": 6}],
        [["line", p2, p3], {"bc": "optimVert", "leftdomain": 5, "rightdomain": 3}],
        [["line", p3, p4], {"bc": "default", "leftdomain": 2, "rightdomain": 3}],
        [["line", p4, p5], {"bc": "optimVert", "leftdomain": 4, "rightdomain": 3}],
        [["line", p5, p6], {"bc": "front", "leftdomain": 4, "rightdomain": 7}],
        [["line", p6, p002], {"bc": "optimVert", "leftdomain": 4, "rightdomain": 1}],
        [["line", p002, p7], {"bc": "default", "leftdomain": 2, "rightdomain": 1}],
        [["line", p7, p8], {"bc": "default", "leftdomain": 2, "rightdomain": 1}],
        [["line", p00, p03], {"bc": "domainHor", "leftdomain": 6, "rightdomain": 0}],
        [["line", p03, p04], {"bc": "segment1", "leftdomain": 3, "rightdomain": 0}],
        [["line", p04, p001], {"bc": "domainHor", "leftdomain": 7, "rightdomain": 0}],
        [["line", p001, p01], {"bc": "segment1", "leftdomain": 1, "rightdomain": 0}],
        [["line", p02, p8], {"bc": "segment2", "leftdomain": 1, "rightdomain": 0}],
        [["line", p8, p003], {"bc": "segment2", "leftdomain": 2, "rightdomain": 0}],
        [["line", p003, p1], {"bc": "domainVert", "leftdomain": 5, "rightdomain": 0}],
        [["line", p1, p00], {"bc": "domainVert", "leftdomain": 6, "rightdomain": 0}],
        [["line", p001, p6], {"bc": "optimVert", "leftdomain": 7, "rightdomain": 1}],
        [["line", p04, p5], {"bc": "optimVert", "leftdomain": 3, "rightdomain": 7}],
        [["line", p2, p03], {"bc": "optimVert", "leftdomain": 3, "rightdomain": 6}],
        [["line", p3, p003], {"bc": "optimHor", "leftdomain": 5, "rightdomain": 2}],
        [["line", p002, p4], {"bc": "optimHor", "leftdomain": 4, "rightdomain": 2}],
        [["spline3", p01, p05, p02], {"bc": "arc", "leftdomain": 1, "rightdomain": 0}],
    ]

    # Append all lines to the geometry
    for line, props in lines:
        geo.Append(line, **props)

    # Debugging with matplotlib
    if debug:
        line_labels_offset = 6e-4
        debug_geometry(geo, line_labels_offset)

    # Set materials and meshing parameters
    geo.SetMaterial(1, "air")
    geo.SetMaterial(2, "core")
    geo.SetMaterial(3, "coil")
    geo.SetMaterial(4, "core")
    geo.SetMaterial(5, "core")
    geo.SetMaterial(6, "air")
    geo.SetMaterial(7, "air")
    ngmesh = geo.GenerateMesh(maxh=maxh)
    return ngs.Mesh(ngmesh)


# def gen_mesh(air_gap, maxh=7e-4, debug=False):
#     """Gives a triangular mesh"""
#     geo = SplineGeometry()
#     pnts = [
#         (0, air_gap / 2),  # p1
#         (a / 2, air_gap / 2),  # p2
#         (a / 2, e / 2 + ha / 2),  # p3
#         (a / 2 + ba, e / 2 + ha / 2),  # p4
#         (a / 2 + ba, air_gap / 2),  # p5
#         (a + ba, air_gap / 2),  # p6
#         (a + ba, e / 2 + ha / 2 + a / 2),  # p7
#         (0, e / 2 + ha / 2 + a / 2),  # p8
#         (a + ba, 0),  # p001
#         (a + ba, e / 2 + ha / 2),  # p002
#         (0, e / 2 + ha / 2),  # p003
#         (0, 0),  # p00
#         (r, 0),  # p01
#         (0, r),  # p02
#         (a / 2, 0),  # p03
#         (a / 2 + ba, 0),  # p04
#         (r, r),  # p05
#     ]

#     (
#         p1,
#         p2,
#         p3,
#         p4,
#         p5,
#         p6,
#         p7,
#         p8,
#         p001,
#         p002,
#         p003,
#         p00,
#         p01,
#         p02,
#         p03,
#         p04,
#         p05,
#     ) = [geo.AppendPoint(*pnt) for pnt in pnts]

#     # List of lines with boundary conditions and domains
#     lines = [
#         [["line", p1, p2], {"bc": "front", "leftdomain": 2, "rightdomain": 6}],
#         [["line", p2, p3], {"bc": "optimVert", "leftdomain": 2, "rightdomain": 3}],
#         [["line", p3, p4], {"bc": "default", "leftdomain": 2, "rightdomain": 3}],
#         [["line", p4, p5], {"bc": "optimVert", "leftdomain": 2, "rightdomain": 3}],
#         [["line", p5, p6], {"bc": "front", "leftdomain": 2, "rightdomain": 1}],
#         [["line", p6, p002], {"bc": "front", "leftdomain": 2, "rightdomain": 1}],  # optimVert -> front
#         [["line", p002, p7], {"bc": "front", "leftdomain": 2, "rightdomain": 1}],  # default -> front
#         [["line", p7, p8], {"bc": "front", "leftdomain": 2, "rightdomain": 1}],  # default -> front
#         [["line", p00, p03], {"bc": "domainHor", "leftdomain": 6, "rightdomain": 0}],
#         [["line", p03, p04], {"bc": "segment1", "leftdomain": 3, "rightdomain": 0}],
#         [["line", p04, p001], {"bc": "domainHor", "leftdomain": 1, "rightdomain": 0}],
#         [["line", p001, p01], {"bc": "segment1", "leftdomain": 1, "rightdomain": 0}],
#         [["line", p02, p8], {"bc": "segment2", "leftdomain": 1, "rightdomain": 0}],
#         [["line", p8, p003], {"bc": "optimVert", "leftdomain": 2, "rightdomain": 0}],  # segment2 -> optimVert
#         [["line", p003, p1], {"bc": "domainVert", "leftdomain": 2, "rightdomain": 0}],
#         [["line", p1, p00], {"bc": "domainVert", "leftdomain": 6, "rightdomain": 0}],
#         # [["line", p001, p6], {"bc": "front", "leftdomain": 7, "rightdomain": 1}],  # optimVert -> front
#         [["line", p04, p5], {"bc": "optimVert", "leftdomain": 3, "rightdomain": 1}],
#         [["line", p2, p03], {"bc": "optimVert", "leftdomain": 3, "rightdomain": 6}],
#         # [["line", p3, p003], {"bc": "optimHor", "leftdomain": 5, "rightdomain": 2}],
#         # [["line", p002, p4], {"bc": "optimHor", "leftdomain": 4, "rightdomain": 2}],
#         [["spline3", p01, p05, p02], {"bc": "arc", "leftdomain": 1, "rightdomain": 0}],
#     ]

#     # Append all lines to the geometry
#     for line, props in lines:
#         geo.Append(line, **props)

#     # Debugging with matplotlib
#     if debug:
#         line_labels_offset = 6e-4
#         debug_geometry(geo, line_labels_offset)

#     # Set materials and meshing parameters
#     geo.SetMaterial(1, "air")
#     geo.SetMaterial(2, "core")
#     geo.SetMaterial(3, "coil")
#     geo.SetMaterial(4, "core")
#     geo.SetMaterial(5, "core")
#     geo.SetMaterial(6, "air")
#     geo.SetMaterial(7, "air")
#     ngmesh = geo.GenerateMesh(maxh=maxh)
#     return ngs.Mesh(ngmesh)


def debug_geometry(geo: SplineGeometry, offset=5e-4):
    """Plot the geometry with matplotlib to help debugging"""
    # Plot the points
    points = []
    nb_points = geo.GetNPoints()
    for i in range(nb_points):
        point = geo.GetPoint(i)
        points.append((point[0], point[1]))
    points = np.array(points)

    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], color="blue")

    # Plot the lines and splines and annotate with region labels
    nb_splines = geo.GetNSplines()
    for i in range(nb_splines):
        spline = geo.GetSpline(i)
        p_start, p_end = spline.StartPoint(), spline.EndPoint()
        plt.plot([p_start[0], p_end[0]], [p_start[1], p_end[1]], color="green")

        # Midpoint for labels
        midpoint = ((p_start[0] + p_end[0]) / 2, (p_start[1] + p_end[1]) / 2)
        dx, dy = p_end[0] - p_start[0], p_end[1] - p_start[1]
        normal = np.array([-dy, dx])
        normal_length = np.linalg.norm(normal)
        if normal_length != 0:
            normal = normal / normal_length
        left_domain_position = (midpoint[0] + normal[0] * offset, midpoint[1] + normal[1] * offset)
        right_domain_position = (midpoint[0] - normal[0] * offset, midpoint[1] - normal[1] * offset)
        left_domain_label = spline.leftdom
        right_domain_label = spline.rightdom
        plt.text(
            left_domain_position[0], left_domain_position[1], left_domain_label, color="red", fontsize=10, ha="center"
        )
        plt.text(
            right_domain_position[0],
            right_domain_position[1],
            right_domain_label,
            color="red",
            fontsize=10,
            ha="center",
        )

        normal_start = spline.GetNormal(0)
        tangent_start = (-normal_start[1], normal_start[0])
        normal_end = spline.GetNormal(1)
        tangent_end = (-normal_end[1], normal_end[0])
        determinant = tangent_start[0] * tangent_end[1] - tangent_end[0] * tangent_start[1]
        if np.abs(determinant) > 1e-16:
            print("spline3 not yet supported, displaying as a line")

    plt.grid(True)
    plt.axis("equal")
    plt.show()


XiAir = ngs.CoefficientFunction([1, 0, 0, 0, 0, 1, 1])
XiCore = ngs.CoefficientFunction([0, 1, 0, 1, 1, 0, 0])
XiCoil = ngs.CoefficientFunction([0, 0, 1, 0, 0, 0, 0])

maxh = 5e-4
air_gap = 4e-3
mesh = gen_mesh(air_gap, maxh, debug=True)

Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)

## 2 - Computation of cost function, state (magnetostatic potential), adjoint state and shape derivative

In [ ]:
L_target = 1e-3

mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
mu_coil = mu0

coeff_losses = 2 * d * (1 / mu_coil)

nb_turn = 200  # Number of turn in the coil
Is = 2  # Source current intensity
js = nb_turn * Is / (ba * (e + ha)) * XiCoil  # Source current density


def LossesDensity(a):
    return 4 * d * 0.5 * (1 / mu_coil) * ngs.InnerProduct(ngs.grad(a), ngs.grad(a)) * ngs.dx("coil")


def Losses(a, mesh):
    """Magnetic energy in the coil domain"""
    return ngs.Integrate(
        4 * d * 0.5 * (1 / mu_coil) * ngs.InnerProduct(ngs.grad(a), ngs.grad(a)) * ngs.dx("coil"), mesh
    )


def CostFunction(a, l, b, constraint, mesh):
    return Losses(a, mesh) + l * constraint + 0.5 * b * constraint**2


def Inductance(a, mesh):
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    return ngs.Integrate(2 * 4 * d * 0.5 * rel * ngs.InnerProduct(ngs.grad(a), ngs.grad(a)) / (Is * Is), mesh)


def Constraint(a, mesh):
    return Inductance(a, mesh) - L_target

In [ ]:
def magWeakForm(u, v):
    bf = ngs.grad(v) * 1 / mu_iron * ngs.grad(u) * ngs.dx("core")
    bf += ngs.grad(v) * 1 / mu_coil * ngs.grad(u) * ngs.dx("coil")
    bf += ngs.grad(v) * 1 / mu0 * ngs.grad(u) * ngs.dx("air")
    lf = v * js * ngs.dx("coil")
    return bf, lf


def solveState(fes):
    u, v = fes.TnT()
    bf, f = magWeakForm(u, v)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs())
    gf.vec.data = Kinv * F.vec
    return gf, Kinv

In [ ]:
mesh = gen_mesh(air_gap=4e-3, maxh=5e-4, debug=False)
# mesh = ngs.Mesh("4_optimized_mesh.vol") # Decomment to get optimized shape

fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
VEC = ngs.VectorH1(mesh)

state, Kinv = solveState(fes)

print(f"The inductance is {1e3 * np.absolute(Inductance(state, mesh)):.3f} mH")
print(f"The magnetic energic inside the coil is {1e3*Losses(state, mesh):.3f} mJ")

Draw(state, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
Draw(ngs.Norm(rot(state)), mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

In [ ]:
def solveAdjointLosses(state, Kinv):
    """Solves the adjoint equation for the losses inside the coil"""
    fes = state.space
    u, v = fes.TnT()
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    f = ngs.LinearForm(fes)
    f += -4 * d * rel * ngs.InnerProduct(ngs.grad(state), ngs.grad(v)) * ngs.dx("coil")
    # f += -1 * LossesDensity(state).Diff(state, v)
    f.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv * f.vec
    return gf


def solveAdjointInductance(state, Kinv):
    """Solves the adjoint equation for the inductance"""
    fes = state.space
    v = fes.TestFunction()
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    F = ngs.LinearForm(fes)
    F += -8 * d * rel / (Is * Is) * ngs.InnerProduct(ngs.grad(state), ngs.grad(v)) * ngs.dx
    F.Assemble()
    gfu = ngs.GridFunction(fes)
    gfu.vec.data = Kinv * F.vec
    return gfu


def computeLossesShapeDerivative(VEC, a0, p0):
    """Shape derivative for the complex losses inside the coil"""
    X = VEC.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix

    dLOmega = ngs.LinearForm(VEC)
    dLOmega += -2 * 2 * d * rel * ngs.InnerProduct(ngs.grad(X) * ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    dLOmega += 2 * d * rel * ngs.InnerProduct(ngs.grad(a0), ngs.grad(a0)) * ngs.div(X) * ngs.dx("coil")
    dLOmega += -js * ngs.div(X) * p0 * ngs.dx  # div(js X) = div(X)js + X * grad(js) = div(X)js a.e.
    dLOmega += (
        ngs.InnerProduct(rel * (ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(a0), ngs.grad(p0))
        * ngs.dx
    )
    return dLOmega.Assemble()


def computeLossesShapeDerivativeAuto(VEC, a0, p0):
    """Shape derivative for the complex losses inside the coil"""
    X = VEC.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore

    # Definition of the Lagrangian of the inductance
    L = LossesDensity(a0) + rel * ngs.grad(a0) * ngs.grad(p0) * ngs.dx - js * p0 * ngs.dx

    dLOmega = ngs.LinearForm(VEC)
    dLOmega += L.DiffShape(X)
    dLOmega.Assemble()
    return dLOmega.Assemble()


def computeInductanceShapeDerivative(VEC, a0, p0):
    """Shape derivative for the inductance"""
    X = VEC.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix

    dLOmega = ngs.LinearForm(VEC)
    dLOmega += -2 * 4 * d * rel / (Is * Is) * ngs.InnerProduct(ngs.grad(X) * ngs.grad(a0), ngs.grad(a0)) * ngs.dx
    dLOmega += 4 * d * rel / (Is * Is) * ngs.InnerProduct(ngs.grad(a0), ngs.grad(a0)) * ngs.div(X) * ngs.dx
    dLOmega += -js * ngs.div(X) * p0 * ngs.dx  # div(js X) = div(X)js + X * grad(js) = div(X)js a.e.
    dLOmega += (
        ngs.InnerProduct(rel * (ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans) * ngs.grad(a0), ngs.grad(p0))
        * ngs.dx
    )
    return dLOmega.Assemble()


def InductanceDensity(a):
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    return 2 * 4 * d * 0.5 * rel * ngs.InnerProduct(ngs.grad(a), ngs.grad(a)) / (Is * Is)


def computeInductanceDerivativeAuto(VEC, a0, p0):
    X = VEC.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore

    # Definition of the Lagrangian of the inductance
    L = InductanceDensity(a0) * ngs.dx + rel * ngs.grad(a0) * ngs.grad(p0) * ngs.dx - js * p0 * ngs.dx

    dLOmega = ngs.LinearForm(VEC)
    dLOmega += L.DiffShape(X)
    dLOmega.Assemble()
    return dLOmega.Assemble()


def computeShapeDerivative(mesh, VEC, state, adjoint_losses, adjoint_inductance, l, b):
    """Shape derivative for the cost function"""
    dJOmegaInductance = computeInductanceShapeDerivative(VEC, state, adjoint_inductance)
    constraint = Constraint(state, mesh)
    dJOmega = computeLossesShapeDerivative(VEC, state, adjoint_losses)
    dJOmega.vec.FV().NumPy()[:] += l * dJOmegaInductance.vec.FV().NumPy()[:]
    dJOmega.vec.FV().NumPy()[:] += b * constraint * dJOmegaInductance.vec.FV().NumPy()[:]
    return dJOmega

In [ ]:
mesh = gen_mesh(air_gap=4e-3, maxh=5e-4, debug=False)
# mesh = ngs.Mesh("4_optimized_mesh.vol") # Decomment to get optimized shape

fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
VEC = ngs.VectorH1(mesh)

state, Kinv = solveState(fes)  # Is a GridFunction (field approximated on a fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)

dJOmega = computeShapeDerivative(mesh, VEC, state, adjoint_losses, adjoint_inductance, 1, 1)

# Draw(state, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
# Draw(ngs.Norm(rot(state)), mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
Draw(adjoint_inductance, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
Draw(adjoint_losses.real, mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

## 3 - Computation of the descent direction

In [ ]:
def SolveDeformationEquation(mesh, fX):
    VEC = ngs.VectorH1(
        mesh,
        dirichlet="arc|segment1|segment2|default",
        dirichlety="optimHor|domainHor",
        dirichletx="optimVert|domainVert",
    )
    PHI, X = VEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    # # Linear elasticity-type dot product (maybe try with lame parameter ?)
    # B = ngs.BilinearForm(VEC)
    # B += (
    #     ngs.InnerProduct(0.5 * (ngs.grad(X) + ngs.grad(X).trans), ngs.grad(PHI)) * ngs.dx
    #     + ngs.InnerProduct(X, PHI) * ngs.dx
    # )
    # B.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = -B.mat.Inverse(VEC.FreeDofs()) * fX.vec
    return gfX


# def SolveDeformationEquation(mesh, fX):
#     VEC = ngs.VectorH1(mesh)
#     PHI, X = VEC.TnT()

#     # H1 dot product
#     B = ngs.BilinearForm(VEC)
#     B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
#     B.Assemble()

#     # GridFunctions used as mask for Dirichlet condition
#     gfX_optimzone_horizontal_mask = ngs.GridFunction(VEC)
#     cf = ngs.CoefficientFunction((1, 1))
#     gfX_optimzone_horizontal_mask.Set(cf, definedon=mesh.Boundaries("optimHor|domainHor"))
#     # Draw(gfX_optimzone_horizontal_mask, mesh, 'gfX_optimzone_horizontal_mask', radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

#     gfX_optimzone_vertical_mask = ngs.GridFunction(VEC)
#     cf = ngs.CoefficientFunction((1, 1))
#     gfX_optimzone_vertical_mask.Set(cf, definedon=mesh.Boundaries("optimVert|domainVert"))
#     # Draw(gfX_optimzone_vertical_mask, mesh, 'gfX_optimzone_vertical_mask', radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

#     gfX_front_mask = ngs.GridFunction(VEC)
#     cf = ngs.CoefficientFunction((1, 1))
#     gfX_front_mask.Set(cf, definedon=mesh.Boundaries("front"))
#     # Draw(gfX_front_mask, mesh, 'gfX_front_mask', radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

#     gfX_outer_mask = ngs.GridFunction(VEC)
#     cf = ngs.CoefficientFunction((1, 1))
#     gfX_outer_mask.Set(cf, definedon=mesh.Boundaries("arc|segment1|segment2|default"))
#     # Draw(gfX_outer_mask, mesh, 'gfX_outer_mask', radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

#     gfX = ngs.GridFunction(VEC)
#     freedofs = VEC.FreeDofs()
#     N = VEC.ndof
#     Nhalf = int(N / 2)

#     for i in range(Nhalf):
#         if ((gfX_optimzone_horizontal_mask.vec[i] > 0.5) or gfX_outer_mask.vec[i] > 0.5) and (
#             gfX_front_mask.vec[i] < 0.5
#         ):
#             freedofs[i] = False
#             freedofs[Nhalf + i] = False
#         elif gfX_optimzone_vertical_mask.vec[i] > 0.5:
#             freedofs[i] = False
#             freedofs[Nhalf + i] = True
#         else:
#             freedofs[i] = True
#             freedofs[Nhalf + i] = True

#     gfX.vec.data = -B.mat.Inverse(freedofs) * fX.vec
#     return gfX

dJOmega = computeLossesShapeDerivative(VEC, state, adjoint_losses)

# Without dirichlet conditions
dJ = ngs.GridFunction(VEC)
dJ.vec.FV().NumPy()[:] = dJOmega.vec.FV().NumPy()[:]
Draw(dJ, mesh, vectors={"grid_size": 40}, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))


# With dirichlet conditions
descent_direction = SolveDeformationEquation(mesh, dJOmega)
Draw(descent_direction, mesh, vectors={"grid_size": 40}, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

In [ ]:
def TaylorTest(cost, compute_adjoint, compute_shape_derivative):
    mesh = gen_mesh(air_gap=4e-3, maxh=3e-4, debug=False)
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=False)
    VEC = ngs.VectorH1(mesh)
    a0, Kinv = solveState(fes)
    p0 = compute_adjoint(a0, Kinv)
    dJOmega = compute_shape_derivative(VEC, a0, p0)

    direction = ngs.GridFunction(VEC)
    direction.Set(ngs.CF((0, 1)), definedon=mesh.Boundaries("front"))

    nb_sample = 20
    exponent_min = 3
    exponent_max = 13

    reminders = []

    X = []

    from mmglib import copy_ngmesh

    meshT = ngs.Mesh(copy_ngmesh(mesh.ngmesh))  # For transported mesh
    fesT = ngs.H1(meshT, order=1, dirichlet="arc|segment2|domainVert", complex=False)
    VECT = ngs.VectorH1(meshT)

    for i in range(nb_sample):
        # Abscissa axis in log scale
        t = 10 ** (-(i / nb_sample * (exponent_max - exponent_min) + exponent_min))
        X.append(t)

        # Finite difference
        displacement = ngs.GridFunction(VECT)
        displacement.Set(t * ngs.CF((0, 1)), definedon=mesh.Boundaries("front"))
        meshT.SetDeformation(displacement)
        aT, _ = solveState(fesT)
        difference = cost(aT, meshT) - cost(a0, mesh)

        # Reminder for analytic shape derivative
        # print(f"{t=}    {difference=}    sensitivity={t * ngs.InnerProduct(dJOmega.vec, direction.vec)}")
        reminder_value = np.abs(difference - t * ngs.InnerProduct(dJOmega.vec, direction.vec))
        reminders.append(reminder_value)

    Y = np.power(np.array(X), 2)
    fig, ax = plt.subplots(1, 1)

    ax.plot(X, reminders, label="reminder", marker="s")
    ax.plot(X, Y, label="y = x^2", linestyle="--", color="gray")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("t")
    ax.set_ylabel("Reminder")
    ax.set_title("Taylor test")
    ax.legend()
    ax.grid(True, which="both", ls="--")

    plt.tight_layout()
    plt.show()


TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative)
# TaylorTest(Inductance, solveAdjointInductance, computeInductanceDerivativeAuto)
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative)
# TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivativeAuto)

In [ ]:
# Initializations
mesh = gen_mesh(air_gap=4e-3, maxh=5e-4, debug=False)
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
VEC = ngs.VectorH1(mesh)

l = 1e-2
b = 1e-1

# State and cost function
state, Kinv = solveState(fes)
constraint = Constraint(state, mesh)
Jold = CostFunction(state, l, b, constraint, mesh)

# Adjoints, shape derivate then descent direction
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)
dJOmega = computeShapeDerivative(mesh, VEC, state, adjoint_losses, adjoint_inductance, 1, 1)

descent_direction = SolveDeformationEquation(mesh, dJOmega)
currentNorm = ngs.Norm(descent_direction.vec)
derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

if derivative > 0:
    raise ValueError("derivative positive :(")


def move_ngmesh_2D(displ, mesh):
    mat_displ = displ.vec.FV().NumPy()
    nb_points = len(mat_displ) // 2
    for i, point in enumerate(mesh.ngmesh.Points()):
        vx = mat_displ[i]
        vy = mat_displ[i + nb_points]
        point[0] += vx
        point[1] += vy
    mesh.ngmesh.Update()


from mmglib import run_adapt, copy_ngmesh

# print("Initial mesh")
# state, Kinv = solveState(fes)
# Draw(ngs.Norm(rot(state)), mesh)

# print("copy of Initial mesh")
# copied_ngmesh = copy_ngmesh(mesh.ngmesh)
# copied_mesh = ngs.Mesh(copied_ngmesh)
# Draw(ngs.Norm(rot(state)), copied_mesh)

print("Moved mesh")
descent_direction.vec.data = 1e-2 * descent_direction.vec
move_ngmesh_2D(descent_direction, mesh)
# mesh.ngmesh.OptimizeMesh2d() # Does not work

# Test the descent of the cost
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
state, Kinv = solveState(fes)
constraint = Constraint(state, mesh)
Jnew = CostFunction(state, l, b, constraint, mesh)

if Jnew < Jold:
    print("J decreased")
else:
    print("J increased")
Draw(ngs.Norm(rot(state)), mesh)

# print("copy of Initial mesh")
# Draw(ngs.Norm(rot(state)), copied_mesh)

print("Optimized moved mesh")
new_ngmesh, return_code = run_adapt(mesh.ngmesh, hausd=3e-6, hmax=2e-3)
new_mesh = ngs.Mesh(new_ngmesh)
fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
state, Kinv = solveState(fes)
Draw(ngs.Norm(rot(state)), new_mesh)

# print("Recover initial mesh")
# descent_direction.vec.data = -descent_direction.vec
# move_ngmesh_2D(descent_direction, mesh)
# fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
# state, tangent_matrix = solveState(fes)
# Draw(ngs.Norm(rot(state)), mesh)

In [ ]:
# Initialize geometry
mesh = gen_mesh(air_gap=4e-3, maxh=5e-4, debug=False)
# mesh = ngs.Mesh("4_optimized_mesh.vol")

fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
VEC = ngs.VectorH1(mesh)

# Initialize data arrays and plots
costs = []
energies = []
inductances = []
num_plots = 3
fig, axes, hdisplay = create_plots(num_plots)
curve_labels = [
    ["Cost Function (W)"],
    ["Losses Conductor (W)"],
    ["Inductance (mH)"],
]

l_values = []
b_values = []
num_plots = 2
figBis, axesBis, hdisplayBis = create_plots(num_plots)
curve_labels_bis = [
    ["l"],
    ["b"],
]

scene = Draw(ngs.Norm(rot(state)), mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

# Algorithmic parameters
iter_max = 300
converged = False
iter = 0
minstep = 1e-10
maxstep = 0.5
step = maxstep
eps = 1e-10

# Augmented Lagragian parameters
l = 1e-2
b = 1e-1
btarget = 1e2

# Initializing state, adjoints, cost function
state, Kinv = solveState(fes)  # Is a GridFunction (field approximated on a fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)
dJOmega = computeShapeDerivative(mesh, VEC, state, adjoint_losses, adjoint_inductance, l, b)
constraint = Constraint(state, mesh)
Jnew = CostFunction(state, l, b, constraint, mesh)

# Optimization loop
while not converged and iter < iter_max:
    # Step 1: Update fespace
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
    VEC = ngs.VectorH1(mesh)

    # Step 2: Update the state and adjoint state
    state, Kinv = solveState(fes)
    adjoint_losses = solveAdjointLosses(state, Kinv)
    adjoint_inductance = solveAdjointInductance(state, Kinv)

    constraintold = Constraint(state, mesh)
    Jold = CostFunction(state, l, b, constraintold, mesh)

    # Step 2.5: Update the data arrays
    costs.append(Jold)
    energies.append(Losses(state, mesh))
    inductances.append(1e3 * Inductance(state, mesh))
    data = [
        [costs],
        [energies],
        [inductances],
    ]
    update_plots(fig, axes, hdisplay, data, curve_labels)

    l_values.append(l)
    b_values.append(b)
    data = [
        [l_values],
        [b_values],
    ]
    update_plots(figBis, axesBis, hdisplayBis, data, curve_labels_bis)

    # Step 3: Compute shape derivative
    dJOmega = computeShapeDerivative(mesh, VEC, state, adjoint_losses, adjoint_inductance, l, b)

    # Step 4: Find a descent direction
    descent_direction = SolveDeformationEquation(mesh, dJOmega)
    # descent_direction.vec.FV().NumPy()[: (VEC.ndof // 2)] = np.zeros(VEC.ndof // 2) # Cancel x components
    currentNorm = ngs.Norm(descent_direction.vec)
    derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

    if derivative > 0:
        raise ValueError(f"derivative positive :( {derivative=}")

    # Step 4.5: Scene redraw
    scene.Redraw(ngs.Norm(rot(state)), mesh, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))
    # scene.Redraw(descent_direction, mesh, vectors={"grid_size": 40}, radius=0.009, center=(a / 2 + ba / 2, (e + ha + a) / 4))

    # Step 5: Find suitable step with a line search
    Jold = Jnew
    copied_ngmesh = copy_ngmesh(mesh.ngmesh)
    copied_mesh = ngs.Mesh(copied_ngmesh)
    descent_direction_old = ngs.GridFunction(VEC)
    descent_direction_old.vec.data = descent_direction.vec
    i = 0
    imax = 7
    while i < imax:
        descent_direction.vec.data = step * maxh / (currentNorm + eps) * descent_direction_old.vec
        move_ngmesh_2D(descent_direction, mesh)

        return_code = 1
        # Mesh adaptation with mmg
        if iter % 10 == 0:
            new_ngmesh, return_code = run_adapt(mesh.ngmesh, hausd=3e-6, hmax=2e-3)
            mesh = ngs.Mesh(new_ngmesh)
            fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert")
            VEC = ngs.VectorH1(mesh)

        # Compute new cost
        state, Kinv = solveState(fes)
        constraint = Constraint(state, mesh)
        Jnew = CostFunction(state, l, b, constraint, mesh)

        if return_code == 1 and Jnew < Jold + 1e-1 * np.abs(Jold):
            step = min(maxstep, 1.2 * step)
            break
        else:
            print(f"Line search fail")
            step = max(minstep, 0.5 * step)
            mesh = copied_mesh
            Jnew = Jold
        i += 1

    # Update augmented lagrangian parameter
    if i == imax:
        # print(f"Line search fail")
        pass
    else:
        # print(f"Line search success")
        l = l + b * Constraint(state, mesh)
        if b < btarget:
            b = min(1.1 * b, btarget)

    # Step 6: Stopping criteria
    if step <= minstep:
        print("Stoping criteria")
        converged = True
    # if np.abs(Jold - Jnew) <= 1e-4 * np.abs(Jold) and np.abs(constraint) <= 0.05 * L_target and b >= btarget:
    #     print("Stoping criteria")
    #     converged = True
    iter += 1

if iter == iter_max:
    print("Gradient descent max iteration reached")

plt.close(fig)

In [ ]:
# mesh.ngmesh.Save("../outputs/4_optimized_mesh.vol")
# fig.savefig("../outputs/4_cv.png")
# figBis.savefig("../outputs/4_cv2.png")
# gf = ngs.GridFunction(ngs.L2(mesh))
# gf.Set(ngs.Norm(rot(state)))
# Draw(gf, mesh, radius=0.01, center=(0, 0), filename="../outputs/4_result.html")

In [ ]:
# materials_list = mesh.GetMaterials()
# boundaries_list = mesh.GetBoundaries()

# new_mesh = ngs.Mesh("../outputs/4_optimized_mesh.vol")

# for i in range(len(materials_list)):
#     new_mesh.ngmesh.SetMaterial(i + 1, materials_list[i])

# for i in range(len(boundaries_list)):
#     new_mesh.ngmesh.SetBCName(i, boundaries_list[i])

# Draw(XiCore + 2 * XiAir + 3 * XiAir, mesh)
# Draw(XiCore + 2 * XiAir + 3 * XiAir, new_mesh)